In [1]:
# Install required packages
!pip install autogen-ext[grpc] gradio -q

# Additional installations for Colab environment
!pip install nest-asyncio -q  # Needed for async in Jupyter


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.70.0 which is incompatible.


In [2]:
!pip install autogen-core autogen-agentchat "autogen-ext[semantic-kernel-google]"

  Using cached grpcio-1.74.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
Using cached grpcio-1.74.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.2 MB)
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.70.0
    Uninstalling grpcio-1.70.0:
      Successfully uninstalled grpcio-1.70.0


In [3]:
# %%
# Import necessary libraries
import asyncio
import json
import logging
import sys
from dataclasses import dataclass
from typing import List, Dict, Optional
from datetime import datetime
import gradio as gr
from IPython.display import display, HTML

# AutoGen imports
from autogen_core import (
    DefaultTopicId,
    MessageContext,
    RoutedAgent,
    default_subscription,
    message_handler
)
from autogen_ext.runtimes.grpc import (
    GrpcWorkerAgentRuntimeHost,
    GrpcWorkerAgentRuntime
)

# Enable async in Jupyter/Colab
import nest_asyncio
nest_asyncio.apply()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ All packages installed successfully!")


# ## Agent Implementation with Proper Communication Setup

from autogen_agentchat.messages import TextMessage
from autogen_core import TopicId, TypeSubscription
import json

# Define a shared topic for communication
TRANSLATION_TOPIC = TopicId("translation_topic", "translation_topic")

✅ All packages installed successfully!


In [4]:
# ## 2. Core Components <a id="components"></a>
#
# Let's define the core message types that our agents will use to communicate.

# %%
# Define message types for agent communication
from dataclasses import dataclass
from typing import Dict
import json

# @dataclass
# class QueryTranslationRequest:
#     """
#     Message sent by RequestAgent to request SQL translation
#     """
#     request_id: str                     # Unique identifier for tracking
#     natural_language_query: str         # The query to translate
#     database_schema_json: str          # Schema information as JSON string
#     sender: str                        # Agent that sent the request
#     timestamp: str                     # When the request was sent

# @dataclass
# class QueryTranslationResponse:
#     """
#     Message sent by TranslatorAgent with SQL translation result
#     """
#     request_id: str      # Matches the request ID
#     sql_query: str       # Generated SQL query
#     confidence: float    # Confidence score (0-1)
#     explanation: str     # Explanation of the translation
#     responder: str       # Agent that created the response
#     timestamp: str       # When the response was sent

print("✅ Message types defined!")

# Message type definitions as string constants
QUERY_TRANSLATION_REQUEST = "QueryTranslationRequest"
QUERY_TRANSLATION_RESPONSE = "QueryTranslationResponse"

print("✅ Message types defined!")

✅ Message types defined!
✅ Message types defined!


In [5]:
# ## 3. Communication Log System <a id="log"></a>
#
# We'll implement a logging system to track all agent communications for visualization.

# %%
# Global communication log to track all messages
communication_log = []

def log_communication(entry_type: str, agent: str, data: dict):
    """
    Add an entry to the communication log
    """
    entry = {
        "type": entry_type,
        "agent": agent,
        "data": data,
        "timestamp": datetime.now().isoformat()
    }
    communication_log.append(entry)

    # Print to console for debugging
    print(f"[{entry_type.upper()}] {agent}: {data.get('status', data.get('query', 'Processing...'))}")

def clear_communication_log():
    """Clear all entries from the communication log"""
    global communication_log
    communication_log = []
    print("✅ Communication log cleared")

# Test the logging system
log_communication("test", "System", {"status": "Logging system initialized"})
print(f"\nCurrent log entries: {len(communication_log)}")

[TEST] System: Logging system initialized

Current log entries: 1


In [6]:
# ## Simplified Cross-Runtime Agent System
# Both agents register on both runtimes to enable communication

from autogen_agentchat.messages import TextMessage
import json
import os

# ### 4.1 Request Agent
@default_subscription
class RequestAgent(RoutedAgent):
    """
    Agent that sends natural language queries for SQL translation
    """

    def __init__(self, name: str) -> None:
        super().__init__(f"RequestAgent-{name}")
        self._name = name
        self._pending_requests = {}  # Track requests awaiting responses

    async def send_translation_request(self, query: str, schema: Dict) -> None:
        """
        Send a query translation request to the network
        """
        # Generate unique request ID
        request_id = f"req_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}"

        # Create request data as JSON
        request_data = {
            "type": "QueryTranslationRequest",
            "request_id": request_id,
            "natural_language_query": query,
            "database_schema": schema,
            "sender": self._name,
            "timestamp": datetime.now().isoformat()
        }

        # Create TextMessage with JSON content
        message = TextMessage(
            content=json.dumps(request_data),
            source=self._name
        )

        # Log the request
        log_communication("request", self._name, {
            "request_id": request_id,
            "query": query,
            "schema": schema
        })
        print(f"\n📤 RequestAgent sending message")

        # Store pending request
        self._pending_requests[request_id] = request_data

        # Send message using default topic
        await self.publish_message(message, DefaultTopicId())

    @message_handler
    async def handle_message(self, message: TextMessage, ctx: MessageContext) -> None:
        """
        Handle incoming messages
        """
        try:
            # Parse the message content
            data = json.loads(message.content)

            # Only process responses, not requests
            if data.get("type") == "QueryTranslationResponse":
                print(f"\n📥 RequestAgent received response from: {message.source}")
                print(f"   Message type: {data.get('type')}")

                request_id = data.get("request_id")
                if request_id in self._pending_requests:
                    # Log the response
                    log_communication("response", self._name, {
                        "request_id": request_id,
                        "sql_query": data.get("sql_query"),
                        "confidence": data.get("confidence"),
                        "explanation": data.get("explanation")
                    })

                    # Remove from pending
                    del self._pending_requests[request_id]
                    print(f"   ✅ Processed response for request: {request_id}")
        except json.JSONDecodeError:
            pass  # Ignore non-JSON messages
        except Exception as e:
            print(f"   Error handling message: {e}")

print("✅ RequestAgent implemented!")


✅ RequestAgent implemented!


In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelFamily

from autogen_core import MessageContext, RoutedAgent, message_handler

In [8]:
from google.colab import userdata

In [9]:
# ### 4.2 Translator Agent
# ## Updated TranslatorAgent with Better API Key Handling
# %%
@default_subscription
class TranslatorAgent(RoutedAgent):
    """
    Agent that translates natural language queries to SQL
    """

    def __init__(self, name: str) -> None:
        super().__init__(f"TranslatorAgent-{name}")
        self._name = name

        self.model_client = OpenAIChatCompletionClient(
            model="gemini-2.0-flash",
            api_key=userdata.get("GOOGLE_API_KEY"),
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
            model_info={
                "family": ModelFamily.GEMINI_2_0_FLASH,
                "function_calling": True,
                "json_output": True,
                "vision": False,
                "structured_output": True,
            }
        )
        print(f"   ✅ Gemini LLM client initialized for {name}")
        # Create a prompt for the LLM
        system_prompt = """You are a SQL expert. Generate SQL queries from natural language descriptions.

        Return your response as a JSON object with the following structure:
        {
            "sql_query": "The SQL query",
            "confidence": 0.0 to 1.0 (how confident you are in the query),
            "explanation": "Brief explanation of what the query does"
        }

        Important rules:
        - Generate valid SQL syntax
        - Use proper JOIN syntax when needed
        - Include appropriate WHERE, GROUP BY, ORDER BY clauses as needed
        - For ambiguous requests, make reasonable assumptions
        - If you cannot generate a valid query, set confidence below 0.5
        """
        self._delegate = AssistantAgent(name, model_client=self.model_client, system_message=system_prompt)

    async def _generate_sql_with_llm(self, query: str, schema: Dict[str, List[Dict[str, str]]], ctx: MessageContext) -> tuple[str, float, str]:
        """
        Generate SQL from natural language using Gemini LLM
        """

        # Format schema information for the prompt
        schema_info = "Database Schema:\n"
        for table_name, columns in schema.items():
            schema_info += f"\nTable: {table_name}\n"
            schema_info += "Columns:\n"
            for col in columns:
                schema_info += f"  - {col['name']}: {col['type']}\n"

        user_prompt = f"""{schema_info}

Natural Language Query: {query}

Generate the SQL query for this request."""

        try:
            print(f"   🤖 Calling Gemini LLM...")
            # Make the API call
            response = await self._delegate.on_messages(
                [TextMessage(content=user_prompt, source="user")],
                ctx.cancellation_token
            )

            print(response.chat_message)
            # Parse the response
            content = response.chat_message.content.replace("```json", "").replace("```", "")
            result = json.loads(content)
            print(result)

            sql_query = result.get("sql_query", "-- Unable to generate SQL")
            confidence = float(result.get("confidence", 0.0))
            explanation = result.get("explanation", "No explanation provided")

            # Validate confidence is between 0 and 1
            confidence = max(0.0, min(1.0, confidence))

            print(f"   ✅ LLM generated SQL successfully")

        except Exception as e:
            # Handle API errors gracefully
            print(f"   ❌ LLM error: {str(e)}")
            sql_query = "-- Error generating SQL query"
            confidence = 0.0
            explanation = f"Error occurred during SQL generation: {str(e)}"

            # Log the error for debugging
            log_communication("error", self._name, {
                "error": str(e),
                "query": query
            })

        return sql_query, confidence, explanation

    @message_handler
    async def handle_message(self, message: TextMessage, ctx: MessageContext) -> None:
        """
        Handle incoming messages
        """
        try:
            # Parse the message content
            data = json.loads(message.content)

            # Only process requests, not responses
            if data.get("type") == "QueryTranslationRequest":
                print(f"\n📥 TranslatorAgent received request from: {message.source}")
                print(f"   Message type: {data.get('type')}")

                request_id = data.get("request_id")
                print(f"   Processing translation request: {request_id}")

                # Log processing start
                log_communication("processing", self._name, {
                    "request_id": request_id,
                    "status": "Translating query..."
                })

                # Generate SQL using the LLM
                sql_query, confidence, explanation = await self._generate_sql_with_llm(
                    data.get("natural_language_query"),
                    data.get("database_schema"),
                    ctx
                )

                # Create response data
                response_data = {
                    "type": "QueryTranslationResponse",
                    "request_id": request_id,
                    "sql_query": sql_query,
                    "confidence": confidence,
                    "explanation": explanation,
                    "responder": self._name,
                    "timestamp": datetime.now().isoformat()
                }

                # Create TextMessage with JSON content
                response = TextMessage(
                    content=json.dumps(response_data),
                    source=self._name
                )

                print(f"   📤 Sending response back")

                # Send response
                await self.publish_message(response, DefaultTopicId())

        except json.JSONDecodeError:
            pass  # Ignore non-JSON messages
        except Exception as e:
            print(f"   Error handling message: {e}")
            import traceback
            traceback.print_exc()

print("✅ Updated TranslatorAgent with better API key handling!")

✅ Updated TranslatorAgent with better API key handling!


In [10]:
# ### 4.3 Message Relay Agent (DEFINE THIS BEFORE USING IT)
# %%
@default_subscription
class MessageRelayAgent(RoutedAgent):
    """
    Agent that relays messages between different runtimes
    """

    def __init__(self, name: str, target_worker=None):
        super().__init__(f"MessageRelay-{name}")
        self._name = name
        self.target_worker = target_worker

    def set_target_worker(self, worker):
        """Set the target worker for relaying messages"""
        self.target_worker = worker

    @message_handler
    async def handle_message(self, message: TextMessage, ctx: MessageContext) -> None:
        """Relay messages to the target worker"""
        if self.target_worker:
            print(f"\n🔄 Relay {self._name} forwarding message")
            await self.target_worker.publish_message(
                message=message,
                topic_id=DefaultTopicId()
            )

print("✅ MessageRelayAgent implemented!")

✅ MessageRelayAgent implemented!


In [11]:
# ## 5. Simplified Agent System Manager
# %%
class AgentSystem:
    """
    Simplified agent system with agents on different ports
    """

    def __init__(self):
        self.host1 = None
        self.host2 = None
        self.worker1 = None
        self.worker2 = None
        self.is_running = False

    async def start(self):
        """Start both agent hosts and workers"""
        try:
            print("🚀 Starting Agent System...")

            # Start Host 1 (Port 50051)
            self.host1 = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
            self.host1.start()
            print("✅ Started Host 1 on localhost:50051")

            # Start Host 2 (Port 50052)
            self.host2 = GrpcWorkerAgentRuntimeHost(address="localhost:50052")
            self.host2.start()
            print("✅ Started Host 2 on localhost:50052")

            # Small delay to ensure hosts are ready
            await asyncio.sleep(1.0)

            # Create Worker 1 connected to Host 1
            self.worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
            await self.worker1.start()
            print("✅ Started Worker 1")

            # Create Worker 2 connected to Host 2
            self.worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50052")
            await self.worker2.start()
            print("✅ Started Worker 2")

            # Register BOTH agents on BOTH workers
            # This enables cross-runtime communication

            # Register on Worker 1
            await RequestAgent.register(
                self.worker1,
                "request_agent",
                lambda: RequestAgent("Agent1")
            )
            await TranslatorAgent.register(
                self.worker1,
                "translator_agent",
                lambda: TranslatorAgent("Agent2")
            )
            print("✅ Registered both agents on Worker 1")

            # Register on Worker 2
            await RequestAgent.register(
                self.worker2,
                "request_agent",
                lambda: RequestAgent("Agent1")
            )
            await TranslatorAgent.register(
                self.worker2,
                "translator_agent",
                lambda: TranslatorAgent("Agent2")
            )
            print("✅ Registered both agents on Worker 2")

            self.is_running = True
            print("\n🎉 Agent System is running!")
            print("   - Host 1 on Port 50051")
            print("   - Host 2 on Port 50052")
            print("   - Both agents registered on both workers for cross-communication")

        except Exception as e:
            print(f"❌ Failed to start agents: {e}")
            import traceback
            traceback.print_exc()
            await self.stop()
            raise

    async def stop(self):
        """Stop all agents and hosts"""
        print("\n🛑 Stopping Agent System...")
        self.is_running = False

        # Stop workers first
        if self.worker1:
            await self.worker1.stop()
            print("✅ Stopped Worker 1")
        if self.worker2:
            await self.worker2.stop()
            print("✅ Stopped Worker 2")

        # Then stop hosts
        if self.host1:
            await self.host1.stop()
            print("✅ Stopped Host 1")
        if self.host2:
            await self.host2.stop()
            print("✅ Stopped Host 2")

        print("✅ All agents stopped")

    async def send_query(self, natural_language_query: str, schema: Dict) -> None:
        """Send a query translation request"""
        if not self.is_running:
            raise RuntimeError("Agent system is not running")

        # Create the request data
        request_data = {
            "type": "QueryTranslationRequest",
            "request_id": f"req_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}",
            "natural_language_query": natural_language_query,
            "database_schema": schema,
            "sender": "system",
            "timestamp": datetime.now().isoformat()
        }

        # Create TextMessage with JSON content
        message = TextMessage(
            content=json.dumps(request_data),
            source="system"
        )

        # Log the request
        log_communication("request", "system", {
            "request_id": request_data["request_id"],
            "query": natural_language_query,
            "schema": schema
        })

        print(f"\n📤 System sending query")

        # Send message through both workers to ensure delivery
        await self.worker1.publish_message(
            message=message,
            topic_id=DefaultTopicId()
        )
        await self.worker2.publish_message(
            message=message,
            topic_id=DefaultTopicId()
        )

# Create global agent system instance
agent_system = AgentSystem()
print("✅ Agent System manager created!")


✅ Agent System manager created!


In [12]:
# ## 6. Test Function
async def test_system():
    """Test the agent communication system"""
    sample_schema = {
        "customers": {
            "id": "integer",
            "name": "varchar(100)",
            "email": "varchar(100)"
        },
        "orders": {
            "id": "integer",
            "customer_id": "integer",
            "amount": "decimal(10,2)"
        }
    }

    try:
        await agent_system.start()
        await asyncio.sleep(1)

        print("\n" + "="*50)
        print("🧪 TESTING AGENT COMMUNICATION")
        print("="*50)

        # Send test query
        await agent_system.send_query("Show customers with their order counts", sample_schema)
        await asyncio.sleep(3)  # Wait for response

        print("\n✅ Test completed!")

    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        await agent_system.stop()

# Run: await test_system()

# ## Quick Test Function
async def quick_api_test():
    """Quick test to verify API key is working"""
    print("\n=== Quick API Test ===")

    # Create a test translator
    test_translator = TranslatorAgent("TestAgent")

    # Try to generate SQL
    sql, conf, exp = await test_translator._generate_sql_with_llm(
        "Show all customers",
        {"customers": {"id": "int", "name": "varchar"}}
    )

    print(f"\nGenerated SQL: {sql}")
    print(f"Confidence: {conf}")
    print(f"Explanation: {exp}")

# Run this to test: await quick_api_test()

In [13]:
# ## 6. Sample Database Schema <a id="schema"></a>
#
# Let's define a sample e-commerce database schema for testing.

# %%
# Sample database schema
SAMPLE_SCHEMA = {
    "customers": [
        {"name": "id", "type": "INTEGER PRIMARY KEY"},
        {"name": "name", "type": "VARCHAR(255)"},
        {"name": "email", "type": "VARCHAR(255)"},
        {"name": "created_at", "type": "TIMESTAMP"},
        {"name": "city", "type": "VARCHAR(100)"},
        {"name": "country", "type": "VARCHAR(100)"}
    ],
    "orders": [
        {"name": "id", "type": "INTEGER PRIMARY KEY"},
        {"name": "customer_id", "type": "INTEGER REFERENCES customers(id)"},
        {"name": "amount", "type": "DECIMAL(10,2)"},
        {"name": "order_date", "type": "TIMESTAMP"},
        {"name": "status", "type": "VARCHAR(50)"},
        {"name": "payment_method", "type": "VARCHAR(50)"}
    ],
    "products": [
        {"name": "id", "type": "INTEGER PRIMARY KEY"},
        {"name": "name", "type": "VARCHAR(255)"},
        {"name": "category", "type": "VARCHAR(100)"},
        {"name": "price", "type": "DECIMAL(10,2)"},
        {"name": "stock", "type": "INTEGER"},
        {"name": "description", "type": "TEXT"}
    ],
    "order_items": [
        {"name": "id", "type": "INTEGER PRIMARY KEY"},
        {"name": "order_id", "type": "INTEGER REFERENCES orders(id)"},
        {"name": "product_id", "type": "INTEGER REFERENCES products(id)"},
        {"name": "quantity", "type": "INTEGER"},
        {"name": "unit_price", "type": "DECIMAL(10,2)"}
    ]
}

# Display schema nicely
def display_schema(schema):
    """Display database schema in a readable format"""
    for table_name, columns in schema.items():
        print(f"\n📊 Table: {table_name}")
        for col in columns:
            print(f"   - {col['name']}: {col['type']}")

display_schema(SAMPLE_SCHEMA)



📊 Table: customers
   - id: INTEGER PRIMARY KEY
   - name: VARCHAR(255)
   - email: VARCHAR(255)
   - created_at: TIMESTAMP
   - city: VARCHAR(100)
   - country: VARCHAR(100)

📊 Table: orders
   - id: INTEGER PRIMARY KEY
   - customer_id: INTEGER REFERENCES customers(id)
   - amount: DECIMAL(10,2)
   - order_date: TIMESTAMP
   - status: VARCHAR(50)
   - payment_method: VARCHAR(50)

📊 Table: products
   - id: INTEGER PRIMARY KEY
   - name: VARCHAR(255)
   - category: VARCHAR(100)
   - price: DECIMAL(10,2)
   - stock: INTEGER
   - description: TEXT

📊 Table: order_items
   - id: INTEGER PRIMARY KEY
   - order_id: INTEGER REFERENCES orders(id)
   - product_id: INTEGER REFERENCES products(id)
   - quantity: INTEGER
   - unit_price: DECIMAL(10,2)


In [14]:
# ## 7. Gradio Interface Implementation <a id="interface"></a>
#
# Now let's create an interactive Gradio interface for our system.

# %%
# Helper functions for Gradio interface

async def start_agents_handler():
    """Start the agent system"""
    try:
        clear_communication_log()
        await agent_system.start()
        return "✅ Agent system started successfully!", get_communication_log_html()
    except Exception as e:
        return f"❌ Failed to start: {str(e)}", get_communication_log_html()

async def stop_agents_handler():
    """Stop the agent system"""
    try:
        await agent_system.stop()
        return "⏹️ Agent system stopped", get_communication_log_html()
    except Exception as e:
        return f"❌ Failed to stop: {str(e)}", get_communication_log_html()

async def send_query_handler(query, custom_schema):
    """Send a translation request"""
    if not agent_system.is_running:
        return "❌ Agent system is not running. Please start it first.", get_communication_log_html()

    # Parse custom schema if provided
    schema = SAMPLE_SCHEMA
    if custom_schema and custom_schema.strip():
        try:
            schema = json.loads(custom_schema)
        except json.JSONDecodeError:
            return "❌ Invalid schema JSON format", get_communication_log_html()

    try:
        await agent_system.send_query(query, schema)
        # Wait for response
        await asyncio.sleep(2.5)
        return f"✅ Query sent: {query}", get_communication_log_html()
    except Exception as e:
        return f"❌ Error: {str(e)}", get_communication_log_html()

def get_communication_log_html():
    """Format communication log as HTML for display"""
    html = "<div style='font-family: monospace; font-size: 12px;'>"

    # Show last 20 entries
    for entry in communication_log[-20:]:
        timestamp = entry['timestamp'].split('T')[1].split('.')[0]

        if entry['type'] == 'request':
            html += f"""
            <div style='background: #e3f2fd; padding: 10px; margin: 5px; border-radius: 5px; border-left: 4px solid #2196F3;'>
                <strong>📤 REQUEST</strong> from {entry['agent']} at {timestamp}<br>
                <strong>Query:</strong> "{entry['data']['query']}"<br>
                <strong>Request ID:</strong> {entry['data']['request_id']}<br>
                <strong>Tables:</strong> {', '.join(entry['data']['schema'].keys())}
            </div>
            """
        elif entry['type'] == 'processing':
            html += f"""
            <div style='background: #fff3e0; padding: 10px; margin: 5px; border-radius: 5px; border-left: 4px solid #FF9800;'>
                <strong>⚙️ PROCESSING</strong> by {entry['agent']} at {timestamp}<br>
                {entry['data']['status']}
            </div>
            """
        elif entry['type'] == 'response':
            html += f"""
            <div style='background: #e8f5e9; padding: 10px; margin: 5px; border-radius: 5px; border-left: 4px solid #4CAF50;'>
                <strong>📥 RESPONSE</strong> to {entry['agent']} at {timestamp}<br>
                <strong>SQL Query:</strong><br>
                <pre style='background: #f5f5f5; padding: 8px; margin: 5px 0; border-radius: 3px; overflow-x: auto;'>{entry['data']['sql_query']}</pre>
                <strong>Confidence:</strong> {entry['data']['confidence']:.1%}<br>
                <strong>Explanation:</strong> {entry['data']['explanation']}
            </div>
            """

    if not communication_log:
        html += "<p style='color: #666;'>No communication yet...</p>"

    html += "</div>"
    return html

print("✅ Gradio helper functions ready!")

✅ Gradio helper functions ready!


In [15]:
# ## 8. Create and Launch Gradio Interface <a id="gradio"></a>

# %%
# Create Gradio interface
def create_gradio_interface():
    """Create the Gradio web interface"""

    with gr.Blocks(title="AutoGen SQL Translation System", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🚀 AutoGen SQL Translation System with gRPC Communication

        This demo showcases a distributed AI agent system using Microsoft's AutoGen framework:

        - **Agent 1** (Port 50051): Sends natural language queries for SQL translation
        - **Agent 2** (Port 50052): Uses LLM-like logic to translate queries to SQL
        - **Communication**: Agents communicate via gRPC protocol

        ---
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 🎛️ System Control")

                with gr.Row():
                    start_btn = gr.Button("🚀 Start Agents", variant="primary", scale=1)
                    stop_btn = gr.Button("⏹️ Stop Agents", variant="stop", scale=1)

                status = gr.Textbox(
                    label="System Status",
                    value="System not started",
                    lines=1
                )

                gr.Markdown("### 📝 Query Translation")

                query_input = gr.Textbox(
                    label="Natural Language Query",
                    placeholder="e.g., Show me all customers who have placed orders",
                    lines=3
                )

                # Example queries
                gr.Examples(
                    examples=[
                        "Show me all customers",
                        "Count total orders",
                        "Get total order amount",
                        "Show customers with their order counts",
                        "List top customers by spending",
                        "Show recent orders",
                        "Find customers with email addresses",
                        "Show all products with low stock"
                    ],
                    inputs=query_input,
                    label="Example Queries"
                )

                with gr.Accordion("Advanced Options", open=False):
                    schema_input = gr.Code(
                        label="Custom Schema (JSON)",
                        language="json",
                        value="",
                        lines=10
                    )

                    gr.Markdown("""
                    **Note:** Leave empty to use the default e-commerce schema.

                    Example custom schema:
                    ```json
                    {
                        "users": [
                            {"name": "id", "type": "INTEGER"},
                            {"name": "username", "type": "VARCHAR(50)"}
                        ]
                    }
                    ```
                    """)

                send_btn = gr.Button("📤 Send Query", variant="primary", size="lg")

            with gr.Column(scale=2):
                gr.Markdown("### 📡 Agent Communication Log")

                comm_log = gr.HTML(
                    label="Communication Log",
                    value=get_communication_log_html()
                )

                with gr.Row():
                    refresh_btn = gr.Button("🔄 Refresh Log", scale=1)
                    clear_btn = gr.Button("🗑️ Clear Log", scale=1)

                gr.Markdown("""
                **Legend:**
                - 📤 Blue: Request sent by Agent 1
                - ⚙️ Orange: Processing by Agent 2
                - 📥 Green: Response received by Agent 1
                """)

        # Event handlers
        start_btn.click(
            fn=start_agents_handler,
            outputs=[status, comm_log]
        )

        stop_btn.click(
            fn=stop_agents_handler,
            outputs=[status, comm_log]
        )

        send_btn.click(
            fn=send_query_handler,
            inputs=[query_input, schema_input],
            outputs=[status, comm_log]
        )

        refresh_btn.click(
            fn=lambda: ("Current status", get_communication_log_html()),
            outputs=[status, comm_log]
        )

        clear_btn.click(
            fn=lambda: (clear_communication_log(), "Log cleared", get_communication_log_html()),
            outputs=[status, comm_log]
        )

    return demo

# Create the interface
demo = create_gradio_interface()
print("✅ Gradio interface created!")


✅ Gradio interface created!


In [ ]:
# ## 9. Launch the Application <a id="launch"></a>
#
# Now let's launch the Gradio interface. In Google Colab, this will create a public URL.

# %%
# Launch the Gradio interface
print("🚀 Launching Gradio interface...")
print("Please wait for the public URL to appear below...\n")

# Launch with public link for Colab
demo.queue().launch(share=True, debug=True)


🚀 Launching Gradio interface...
Please wait for the public URL to appear below...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a18be52e7bd0c7c0ed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Communication log cleared
🚀 Starting Agent System...
✅ Started Host 1 on localhost:50051
✅ Started Host 2 on localhost:50052
✅ Started Worker 1
✅ Started Worker 2
✅ Registered both agents on Worker 1
✅ Registered both agents on Worker 2

🎉 Agent System is running!
   - Host 1 on Port 50051
   - Host 2 on Port 50052
   - Both agents registered on both workers for cross-communication
[REQUEST] system: Show me all customers

📤 System sending query
   ✅ Gemini LLM client initialized for Agent2

📥 TranslatorAgent received request from: system
   Message type: QueryTranslationRequest
   Processing translation request: req_20250726_225019_020876
[PROCESSING] Agent2: Translating query...
   🤖 Calling Gemini LLM...
   ✅ Gemini LLM client initialized for Agent2

📥 TranslatorAgent received request from: system
   Message type: QueryTranslationRequest
   Processing translation request: req_20250726_225019_020876
[PROCESSING] Agent2: Translating query...
   🤖 Calling Gemini LLM...
id='a70b41f9-c2